In [2]:
import pandas as pd
from scripts.utils import class_count
df = pd.read_csv("data/concatenated/X_balanced.csv")
p,s,w = class_count(df)

Examples:
    Total: 224092
    P: 80056 (35.72% of total)
    S: 64346 (28.71% of total)
    W: 79690 (35.56% of total)



In [5]:
df = pd.read_csv("data/concatenated/window_balanced.csv")
p,s,w = class_count(df)

Examples:
    Total: 224235
    P: 80224 (35.78% of total)
    S: 64539 (28.78% of total)
    W: 79472 (35.44% of total)

